# 通道

通道提供了一种机制，用于通过 发送 和 接收 指定 元素类型 的值来并发执行函数以进行通信。

通道类型语法：`chan<- 元素类型`（仅能发送），`<-chan 元素类型`（仅能接收），`chan 元素类型`（双向）。

    chan T          // can be used to send and receive values of type T
    chan<- float64  // can only be used to send float64s
    <-chan int      // can only be used to receive ints
    
声明一个仅能发送 int 类型的值的通道类型变量如下：

In [3]:
var ch chan<- int
ch // 未初始化通道，值为 nil

<nil>

In [4]:
import "fmt"
fmt.Sprintf("%T",ch) // 通道的类型

chan<- int

通道的 `元素类型` 也可以是通道：

    chan<- chan int    // same as chan<- (chan int)
    chan<- <-chan int  // same as chan<- (<-chan int)
    <-chan <-chan int  // same as <-chan (<-chan int)

可以使用内置函数 make 创建一个新的初始化通道，该函数将通道类型和可选 容量 作为参数。例如：

In [6]:
make(chan int, 100)

0xc0001c8000

容量根据元素的数量设置通道中缓冲区的大小。如果容量为零或不存在，则通道无缓冲，并且仅当发送方和接收方都准备就绪时，通信才会成功。否则，通道将被缓冲，如果缓冲区未满（发送）或不为空（接收），则通信成功而不会阻塞。nil 通道永远不会准备好进行通信。

**发送** 与 **接收**：

发送语句在通道上发送值，语法：`通道 <- 表达式`，表示用 `通道` 发送 `表达式` 的值。通道方向必须允许发送操作，并且要发送的值的类型必须可分配给通道的 `元素类型`。

接收操作 `<- 通道`，表示接收通道的（下一个）值，当前值会被丢弃：

In [7]:
import "fmt"

func sendData(ch chan string) {
        ch <- "OK"
}

func getData(ch chan string) {
        input := <- ch
        fmt.Printf("%s", input)
}

In [10]:
import "time"

ch := make(chan string) // 通道 ch

go sendData(ch)
go getData(ch)
time.Sleep(1e9) // 等待了 1 秒让两个协程完成，如果注释掉将不会打印

OK

只接收的通道无法关闭，因为关闭通道是发送者用来表示不再给通道发送值了，所以对只接收通道是没有意义的。通道可通过内建函数 close 关闭；接收操作符的多值赋值形式可测试通道是否关闭：

In [17]:
import "fmt"

func sendData(ch chan string) {
    ch <- "Washington"
    ch <- "Tripoli"
    ch <- "London"
    ch <- "Beijing"
    ch <- "Tokio"
    close(ch)
}

func getData(ch chan string) {
    for {
        input, open := <- ch
        if !open {
            break
        }
        fmt.Printf("%s\n", input)
    }
}

In [18]:
ch := make(chan string)
go sendData(ch)
getData(ch)

Washington
Tripoli
London
Beijing
Tokio


In [2]:
import "fmt"

queue := make(chan string, 2)
queue <- "one"
queue <- "two"
close(queue)

for elem := range queue {
    fmt.Println(elem)
}

one
two
